# Data cleaning with Spark

Obtained the data from the OSMI website https://osmihelp.org/research.

In [1]:
import pyspark
import pickle 
import pandas as pd
from pyspark.sql.functions import when,udf
from pyspark.sql.types import BooleanType
from pyspark.sql import functions as f
from pyspark.sql.types import *

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [3]:
data_2019 = spark.read.csv(r'/Users/shiehan/Desktop/kaplan stuff/project5/OSMI 2019 Mental Health in Tech Survey Results - OSMI Mental Health in Tech Survey 2019.csv', header = 'true',inferSchema='true',sep=',')
data_2018 = spark.read.csv(r'/Users/shiehan/Desktop/kaplan stuff/project5/OSMI Mental Health in Tech Survey 2018.csv', header = 'true',inferSchema='true',sep=',')
data_2017 = spark.read.csv(r'/Users/shiehan/Desktop/kaplan stuff/project5/OSMI Mental Health in Tech Survey 2017.csv', header = 'true',inferSchema='true',sep=',')
data_2016 = spark.read.csv(r'/Users/shiehan/Desktop/kaplan stuff/project5/mental-heath-in-tech-2016_20161114.csv', header = 'true',inferSchema='true',sep=',')

In [4]:
data_2018=data_2018.drop('#')
data_2018.take(5)

[Row(<strong>Are you self-employed?</strong>='0', How many employees does your company or organization have?='More than 1000', Is your employer primarily a tech company/organization?='1', Is your primary role within your company related to tech/IT?='0', Does your employer provide mental health benefits as part of healthcare coverage?='Yes', Do you know the options for mental health care available under your employer-provided health coverage?='Yes', Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?='Yes', Does your employer offer resources to learn more about mental health disorders and options for seeking help?='Yes', Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?='Yes', If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?='Somew

In [5]:
data_2017=data_2017.drop('#')
data_2017.printSchema()

root
 |-- <strong>Are you self-employed?</strong>: string (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = true)
 |-- Is your employer primarily a tech company/organization?: string (nullable = true)
 |-- Is your primary role within your company related to tech/IT?: string (nullable = true)
 |-- Does your employer provide mental health benefits as part of healthcare coverage?: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided health coverage?: string (nullable = true)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = true)
 |-- Does your employer offer resources to learn more about mental health disorders and options for seeking help?: string (nullable = true)
 |-- Is your anonymity protected if you choose to take advantage of mental health or substance abuse t

In [6]:
# change question name in 2019 to match with the same question in other surveys
data_2019 = data_2019.withColumnRenamed("Have you ever been *diagnosed* with a mental health disorder?","Have you ever been diagnosed with a mental health disorder?").withColumnRenamed("Do you *currently* have a mental health disorder?","Do you currently have a mental health disorder?")

In [7]:
data_2016.printSchema()

root
 |-- Are you self-employed?: string (nullable = true)
 |-- How many employees does your company or organization have?: string (nullable = true)
 |-- Is your employer primarily a tech company/organization?: string (nullable = true)
 |-- Is your primary role within your company related to tech/IT?: string (nullable = true)
 |-- Does your employer provide mental health benefits as part of healthcare coverage?: string (nullable = true)
 |-- Do you know the options for mental health care available under your employer-provided coverage?: string (nullable = true)
 |-- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?: string (nullable = true)
 |-- Does your employer offer resources to learn more about mental health concerns and options for seeking help?: string (nullable = true)
 |-- Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provid

In [8]:
# change question name in 2016 to match with the same question in other surveys
data_2016 = data_2016.withColumnRenamed("Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?","Would you have been willing to discuss your mental health with your direct supervisor(s)?")\
.withColumnRenamed("Do you know the options for mental health care available under your employer-provided coverage?","Do you know the options for mental health care available under your employer-provided health coverage?")\
.withColumnRenamed("Did your previous employers provide resources to learn more about mental health issues and how to seek help?","Did your previous employers provide resources to learn more about mental health disorders and how to seek help?")\
.withColumnRenamed("Do you have medical coverage (private insurance or state-provided) which includes treatment of  mental health issues?","Do you have medical coverage (private insurance or state-provided) that includes treatment of mental health disorders?")\
.withColumnRenamed("Have you ever sought treatment for a mental health issue from a mental health professional?","Have you ever sought treatment for a mental health disorder from a mental health professional?")\
.withColumnRenamed("Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?","Would you feel comfortable discussing a mental health issue with your direct supervisor(s)?")\
.withColumnRenamed("Have you been diagnosed with a mental health condition by a medical professional?","Have you ever been diagnosed with a mental health disorder?")\
.withColumnRenamed("Do you feel that being identified as a person with a mental health issue would hurt your career?","Has being identified as a person with a mental health issue affected your career?")\
.withColumnRenamed("Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?","Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?")\



In [9]:
# find out common questions in all surveys
common_set = list(set(data_2019.columns).intersection(set(data_2018.columns))\
     .intersection(set(data_2017.columns)).intersection(set(data_2016.columns)))

In [10]:
data_2019 = data_2019.select(*common_set)
data_2018 = data_2018.select(*common_set)
data_2017 = data_2017.select(*common_set)
data_2016 = data_2016.select(*common_set)

In [11]:
data = data_2019.union(data_2018).union(data_2017).union(data_2016)

In [13]:
# write a function cast the column to integer
def is_digit(value):
    if value:
        return value.isdigit()
    else:
        return False

is_digit_udf = udf(is_digit, BooleanType())

In [14]:
# group the nulls and unreasonable responses to the 'Other' category
data = data.withColumn('Have you had a mental health disorder in the past?',f.when(~data['Have you had a mental health disorder in the past?'].isin('Possibly', 'No', 'Yes', 'Maybe'),'Other').otherwise(data['Have you had a mental health disorder in the past?']))
data = data.withColumn('Is your employer primarily a tech company/organization?',f.when(~data['Is your employer primarily a tech company/organization?'].isin('1', '0','TRUE','FALSE'),'Other').otherwise(data['Is your employer primarily a tech company/organization?']))
data = data.withColumn('If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',f.when(~data['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?'].isin('1-25%', '76-100%', '26-50%','51-75%'),'Other').otherwise(data['If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?']))
data = data.withColumn('Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',f.when(~data['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?'].isin('Yes', 'No'),'Other').otherwise(data['Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?']))
data = data.withColumn('Would you be willing to bring up a physical health issue with a potential employer in an interview?',f.when(~data['Would you be willing to bring up a physical health issue with a potential employer in an interview?'].isin('Yes', 'No', 'Maybe'),'Other').otherwise(data['Would you be willing to bring up a physical health issue with a potential employer in an interview?']))
data = data.withColumn('Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',f.when(~data['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?'].isin("I don't know", 'Yes','No'),'Other').otherwise(data['Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?']))
data = data.withColumn('What is your gender?',f.when(~data['What is your gender?'].isin('Male','male','Female','female','M','m','F','f','Male','Female','Woman'),'Other').otherwise(data['What is your gender?']))
data = data.withColumn('Did your previous employers provide resources to learn more about mental health disorders and how to seek help?',f.when(~data['Did your previous employers provide resources to learn more about mental health disorders and how to seek help?'].isin('Yes, they all did', 'Some did', 'None did'),'Other').otherwise(data['Did your previous employers provide resources to learn more about mental health disorders and how to seek help?']))
data = data.withColumn('Do you have a family history of mental illness?',f.when(~data['Do you have a family history of mental illness?'].isin('Yes','No',"I don't know"),'Other').otherwise(data['Do you have a family history of mental illness?']))
data = data.withColumn('Do you believe your productivity is ever affected by a mental health issue?',f.when(~data['Do you believe your productivity is ever affected by a mental health issue?'].isin('Yes','Unsure','Not applicable to me'),'Other').otherwise(data['Do you believe your productivity is ever affected by a mental health issue?']))
data = data.withColumn('Would you have been willing to discuss your mental health with your direct supervisor(s)?',f.when(~data['Would you have been willing to discuss your mental health with your direct supervisor(s)?'].isin('Some of my previous supervisors','No, none of my previous supervisors','Yes','Maybe','No',"I don't know",'Yes, all of my previous supervisors'),'Other').otherwise(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?']))
data = data.withColumn('How many employees does your company or organization have?',f.when(~data['How many employees does your company or organization have?'].isin('26-100', '100-500','6-25', 'More than 1000','500-1000','1-5'),'Other').otherwise(data['How many employees does your company or organization have?']))
data = data.withColumn('Have you ever sought treatment for a mental health disorder from a mental health professional?',f.when(~data['Have you ever sought treatment for a mental health disorder from a mental health professional?'].isin('FALSE', 'TRUE','No', 'Yes', '1', '0'),'Other').otherwise(data['Have you ever sought treatment for a mental health disorder from a mental health professional?']))
data = data.withColumn('Do you know the options for mental health care available under your employer-provided health coverage?',f.when(~data['Do you know the options for mental health care available under your employer-provided health coverage?'].isin('No','Yes','I am not sure'),'Other').otherwise(data['Do you know the options for mental health care available under your employer-provided health coverage?']))
data = data.withColumn('Have you ever been diagnosed with a mental health disorder?',f.when(~data['Have you ever been diagnosed with a mental health disorder?'].isin('No','Yes'),'Other').otherwise(data['Have you ever been diagnosed with a mental health disorder?']))
data = data.withColumn('Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',f.when(~data['Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?'].isin('None did','Some did',"I don't know','Yes, they all did"),'Other').otherwise(data['Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?']))
data = data.withColumn('Have you had a mental health disorder in the past?',f.when(~data['Have you had a mental health disorder in the past?'].isin('Yes',"No",'Possibly','Maybe',"Don't Know"),'Other').otherwise(data['Have you had a mental health disorder in the past?']))
data = data.withColumn('Has being identified as a person with a mental health issue affected your career?',f.when(~data['Has being identified as a person with a mental health issue affected your career?'].isin('Maybe','Yes, I think it would',"No, I don't think it would",'0','Yes, it has'),'Other').otherwise(data['Has being identified as a person with a mental health issue affected your career?']))
data = data.withColumn('Is your primary role within your company related to tech/IT?',f.when(~data['Is your primary role within your company related to tech/IT?'].isin('TRUE','1','0','FALSE'),'Other').otherwise(data['Is your primary role within your company related to tech/IT?']))
data = data.withColumn('How willing would you be to share with friends and family that you have a mental illness?',f.when(~data['How willing would you be to share with friends and family that you have a mental illness?'].isin('Somewhat open','Very open','Somewhat not open','Neutral','Not applicable to me (I do not have a mental illness)','Not open at all'),'Other').otherwise(data['How willing would you be to share with friends and family that you have a mental illness?']))
data = data.withColumn('Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?',f.when(~data['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?'].isin("I don't know",'Yes, always','Sometimes','No'),'Other').otherwise(data['Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?']))


In [15]:
data = data.withColumn('What is your age?',f.when(~is_digit_udf(data['What is your age?']),'Other').otherwise(data['What is your age?']))

In [16]:
data = data.withColumn('Do you currently have a mental health disorder?',f.when(~data['Do you currently have a mental health disorder?'].isin('Possibly', 'No', 'Yes', 'Maybe','No'),'Other').otherwise(data['Do you currently have a mental health disorder?']))

In [17]:
# group the similar answers together
data = data.withColumn('Have you had a mental health disorder in the past?',f.when(data['Have you had a mental health disorder in the past?'].isin('Possibly'),'Maybe').otherwise(data['Have you had a mental health disorder in the past?']))

In [18]:
data = data.withColumn('What is your gender?',f.when(data['What is your gender?'].isin('female','Female','F','f','Woman'),'Female').otherwise(data['What is your gender?']))
data = data.withColumn('What is your gender?',f.when(data['What is your gender?'].isin('Male','male','m','M'),'Male').otherwise(data['What is your gender?']))

In [19]:
data = data.withColumn('Do you currently have a mental health disorder?',f.when(data['Do you currently have a mental health disorder?'].isin('Possibly'),'Maybe').otherwise(data['Do you currently have a mental health disorder?']))

In [20]:
data = data.withColumn('Has being identified as a person with a mental health issue affected your career?',f.when(data['Has being identified as a person with a mental health issue affected your career?'].isin('0'),"No, I don't think it would").otherwise(data['Has being identified as a person with a mental health issue affected your career?']))

In [21]:
data = data.withColumn('Have you ever sought treatment for a mental health disorder from a mental health professional?',f.when(data['Have you ever sought treatment for a mental health disorder from a mental health professional?'].isin('FALSE','0'), 'No').otherwise(data['Have you ever sought treatment for a mental health disorder from a mental health professional?']))
data = data.withColumn('Have you ever sought treatment for a mental health disorder from a mental health professional?',f.when(data['Have you ever sought treatment for a mental health disorder from a mental health professional?'].isin('TRUE','1'), 'Yes').otherwise(data['Have you ever sought treatment for a mental health disorder from a mental health professional?']))

In [22]:
data = data.withColumn('Would you have been willing to discuss your mental health with your direct supervisor(s)?',f.when(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?'].isin('No, none of my previous supervisors'), 'No').otherwise(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?']))
data = data.withColumn('Would you have been willing to discuss your mental health with your direct supervisor(s)?',f.when(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?'].isin('Yes, all of my previous supervisors'), 'Yes').otherwise(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?']))
data = data.withColumn('Would you have been willing to discuss your mental health with your direct supervisor(s)?',f.when(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?'].isin('Some of my previous supervisors'), 'Maybe').otherwise(data['Would you have been willing to discuss your mental health with your direct supervisor(s)?']))

In [23]:
data = data.withColumn('Is your employer primarily a tech company/organization?',f.when(data['Is your employer primarily a tech company/organization?'].isin('1'), 'TRUE').otherwise(data['Is your employer primarily a tech company/organization?']))
data = data.withColumn('Is your employer primarily a tech company/organization?',f.when(data['Is your employer primarily a tech company/organization?'].isin('0'), 'FALSE').otherwise(data['Is your employer primarily a tech company/organization?']))


In [24]:
data = data.withColumn('Is your primary role within your company related to tech/IT?',f.when(data['Is your primary role within your company related to tech/IT?'].isin('1'), 'TRUE').otherwise(data['Is your primary role within your company related to tech/IT?']))
data = data.withColumn('Is your primary role within your company related to tech/IT?',f.when(data['Is your primary role within your company related to tech/IT?'].isin('0'), 'FALSE').otherwise(data['Is your primary role within your company related to tech/IT?']))


In [25]:
new_col = ['Q'+str(i) for i in range(1,23)]

In [26]:
data = data.toDF(*new_col)

In [27]:
new_data = data.filter(data['Q19'] != 'Other')

In [28]:
new_data.withColumn("Q19",new_data['Q19'].cast(IntegerType()))

DataFrame[Q1: string, Q2: string, Q3: string, Q4: string, Q5: string, Q6: string, Q7: string, Q8: string, Q9: string, Q10: string, Q11: string, Q12: string, Q13: string, Q14: string, Q15: string, Q16: string, Q17: string, Q18: string, Q19: int, Q20: string, Q21: string, Q22: string]

In [29]:
new_data = new_data.withColumn('Q19',f.when(data['Q19'].isin(0), 34).otherwise(data['Q19']))

In [30]:
def save(clf, name):
    pickle_out = open(name,"wb")
    pickle.dump(clf, pickle_out)
    pickle_out.close()
    print ('Model ',name,' saved')
#save(new_data, 'new_data.pkl')